Table 1.1: Estimated Net International Migration (year ending mid-2001 to year ending mid-2017)

In [1]:
from databaker.framework import *
import pandas as pd 

In [2]:
import requests
from cachecontrol import CacheControl
from cachecontrol.caches.file_cache import FileCache
from cachecontrol.heuristics import LastModified
from pathlib import Path

session = CacheControl(requests.Session(),
                       cache=FileCache('.cache'),
                       heuristic=LastModified())

sourceFolder = Path('in')
sourceFolder.mkdir(exist_ok=True)

inputURL = 'https://www.nisra.gov.uk/sites/nisra.gov.uk/files/publications/Mig1617-Official.xlsx'
inputFile = sourceFolder / 'Mig1617-Official.xlsx'
response = session.get(inputURL)
with open(inputFile, 'wb') as f:
  f.write(response.content)

In [3]:
tab = loadxlstabs(inputFile, sheetids='Table 1.1')[0]

Loading in\Mig1617-Official.xlsx which has size 194382 bytes
Table names: ['Table 1.1']


In [4]:
tab

{<B17 13300.0>, <A14 'Jul 2010 - Jun 2011'>, <B11 18261.0>, <A19 'Jul 2015 - Jun 2016'>, <B13 13877.0>, <B20 11310.0>, <B15 12922.0>, <A20 'Jul 2016 - Jun 2017'>, <B9 15803.0>, <C16 13623.0>, <D11 6280.0>, <D12 2998.0>, <A6 'Jul 2002 - Jun 2003'>, <D21 37116.0>, <C6 6683.0>, <D10 9140.0>, <A18 'Jul 2014 - Jun 2015'>, <E18 'Jul 14 - Jun 15'>, <C19 11540.0>, <E19 'Jul 15 - Jun 16'>, <A8 'Jul 2004 - Jun 2005'>, <E4 'Jul 00 - Jun 01'>, <C7 6996.0>, <C9 8797.0>, <D18 2795.0>, <A5 'Jul 2001 - Jun 2002'>, <D3 'Net International Migration'>, <E7 'Jul 03 - Jun 04'>, <D8 3683.0>, <B2 ''>, <B8 12544.0>, <E14 'Jul 10 - Jun 11'>, <D16 -887.0>, <B7 8174.0>, <C17 11063.0>, <A11 'Jul 2007 - Jun 2008'>, <A2 ''>, <E17 'Jul 13 - Jun 14'>, <B21 212418.0>, <E2 ''>, <A4 'Jul 2000 - Jun 2001'>, <C20 10727.0>, <D15 442.0>, <D4 -1085.0>, <A21 'Total'>, <A17 'Jul 2013 - Jun 2014'>, <B3 'International Inflows'>, <E15 'Jul 11 - Jun 12'>, <D6 127.0>, <C15 12480.0>, <C1 ''>, <C13 12115.0>, <E9 'Jul 05 - Jun 06'>, <

In [5]:
# tab = pd.read_excel(inputFile, header = None, sheet_name = 1)
# tab

In [6]:
observations = tab.excel_ref('B4').expand(DOWN).expand(RIGHT).is_not_blank()- tab.excel_ref('E4').expand(DOWN).expand(RIGHT)  

In [7]:
observations

{<C18 10298.0>, <B17 13300.0>, <D8 3683.0>, <B8 12544.0>, <B18 13093.0>, <B11 18261.0>, <C17 11063.0>, <D16 -887.0>, <B7 8174.0>, <B13 13877.0>, <B20 11310.0>, <B15 12922.0>, <D17 2237.0>, <D19 1458.0>, <C4 7609.0>, <B14 13401.0>, <B21 212418.0>, <C16 13623.0>, <B9 15803.0>, <D11 6280.0>, <C20 10727.0>, <B5 6488.0>, <C10 10633.0>, <C12 11406.0>, <D15 442.0>, <D14 -696.0>, <D4 -1085.0>, <D12 2998.0>, <B6 6810.0>, <D21 37116.0>, <C6 6683.0>, <D10 9140.0>, <C14 14097.0>, <C19 11540.0>, <B10 19773.0>, <D6 127.0>, <B16 12736.0>, <D9 7006.0>, <C5 6393.0>, <B12 14404.0>, <C15 12480.0>, <C13 12115.0>, <B4 6524.0>, <D13 1762.0>, <D20 583.0>, <C7 6996.0>, <C21 175302.0>, <D5 95.0>, <C9 8797.0>, <D18 2795.0>, <C11 11981.0>, <C8 8861.0>, <D7 1178.0>, <B19 12998.0>}

In [8]:
Flow = tab.excel_ref('B3').expand(RIGHT).is_not_blank()
Flow

{<B3 'International Inflows'>, <D3 'Net International Migration'>, <C3 'International Outflows'>}

In [9]:
MidYear = tab.excel_ref('A4').expand(DOWN) - tab.excel_ref('A22').expand(DOWN)  
MidYear

{<A21 'Total'>, <A6 'Jul 2002 - Jun 2003'>, <A14 'Jul 2010 - Jun 2011'>, <A17 'Jul 2013 - Jun 2014'>, <A18 'Jul 2014 - Jun 2015'>, <A8 'Jul 2004 - Jun 2005'>, <A19 'Jul 2015 - Jun 2016'>, <A11 'Jul 2007 - Jun 2008'>, <A7 'Jul 2003 - Jun 2004'>, <A20 'Jul 2016 - Jun 2017'>, <A15 'Jul 2011 - Jun 2012'>, <A9 'Jul 2005 - Jun 2006'>, <A4 'Jul 2000 - Jun 2001'>, <A16 'Jul 2012 - Jun 2013'>, <A13 'Jul 2009 - Jun 2010'>, <A12 'Jul 2008 - Jun 2009'>, <A10 'Jul 2006 - Jun 2007'>, <A5 'Jul 2001 - Jun 2002'>}

In [10]:
Dimensions = [
            HDim(MidYear,'Mid Year',DIRECTLY,LEFT),
            HDim(Flow,'Flow',DIRECTLY, ABOVE),
            HDimConst('Measure Type', 'Count'),
            HDimConst('Unit','People')
            ]

In [11]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# savepreviewhtml(c1)

In [12]:
new_table = c1.topandas()
new_table

,OBS,Mid Year,Flow,Measure Type,Unit
0,6524.0,Jul 2000 - Jun 2001,International Inflows,Count,People
1,7609.0,Jul 2000 - Jun 2001,International Outflows,Count,People
2,-1085.0,Jul 2000 - Jun 2001,Net International Migration,Count,People
3,6488.0,Jul 2001 - Jun 2002,International Inflows,Count,People
4,6393.0,Jul 2001 - Jun 2002,International Outflows,Count,People
5,95.0,Jul 2001 - Jun 2002,Net International Migration,Count,People
6,6810.0,Jul 2002 - Jun 2003,International Inflows,Count,People
7,6683.0,Jul 2002 - Jun 2003,International Outflows,Count,People
8,127.0,Jul 2002 - Jun 2003,Net International Migration,Count,People
9,8174.0,Jul 2003 - Jun 2004,International Inflows,Count,People


In [13]:
new_table['OBS'] = new_table['OBS'].astype(int)

In [14]:
new_table.dtypes

OBS              int32
Mid Year        object
Flow            object
Measure Type    object
Unit            object
dtype: object

In [15]:
new_table['Mid Year'] = new_table['Mid Year'].map(lambda cell:cell.replace('Total', 'all years'))

In [16]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [17]:
new_table['Flow'] = new_table['Flow'].map(
    lambda x: {
        'Net International Migration' : 'Balance', 
        'International Inflows' : 'Inflow',
        'International Outflows': 'Outflow'         
        }.get(x, x))


In [18]:
new_table = new_table[['Mid Year','Flow','Measure Type','Value','Unit']]

In [19]:
new_table.head(5)

,Mid Year,Flow,Measure Type,Value,Unit
0,Jul 2000 - Jun 2001,Inflow,Count,6524,People
1,Jul 2000 - Jun 2001,Outflow,Count,7609,People
2,Jul 2000 - Jun 2001,Balance,Count,-1085,People
3,Jul 2001 - Jun 2002,Inflow,Count,6488,People
4,Jul 2001 - Jun 2002,Outflow,Count,6393,People


In [20]:
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

new_table.to_csv(destinationFolder / ('tidydata1.1.csv'), index = False)